In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import IPython

def display(*dfs):
    for df in dfs:
        IPython.display.display(df)


In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%time df = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv', engine='c')
labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')

In [ ]:
df_test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')

In [ ]:
df = reduce_mem_usage(df)
labels = reduce_mem_usage(labels)
df_test = reduce_mem_usage(df_test)

In [ ]:
train_id = df.installation_id.unique()
test_id = df_test.installation_id.unique()

print('# of unique ids in train:', train_id.shape[0])
print('# of unique ids in test:', test_id.shape[0])
np.isin(test_id, train_id).sum()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

In [ ]:
df.timestamp.describe(), df_test.timestamp.describe()

# Number of attempts

In [ ]:
def event_finished(df):
    df_wt_BM = df[(df.event_code == 4100) & (df.title.str.find('Bird Measurer')==-1)]
    df_BM = df[(df.event_code == 4110) & (df.title.str.find('Bird Measurer')!=-1)]
    return df_wt_BM.append(df_BM)

In [ ]:
assessment = df_test[df_test.type=='Assessment']
assessment_finished = event_finished(assessment)

In [ ]:
print('# ids in test\t # ids in assessment\t # ids in finished assessments')
print(test_id.shape,'\t', assessment.installation_id.unique().shape, '\t\t',
      assessment_finished.installation_id.unique().shape)

In [ ]:
assessment = df[df.type=='Assessment']
assessment_finished = event_finished(assessment)

In [ ]:
print('# ids in train\t # ids in assessment\t # ids in finished assessments')
print(train_id.shape,'\t', assessment.installation_id.unique().shape, '\t\t',
      assessment_finished.installation_id.unique().shape)

For test set - each id took assessments, whereas a lot of ids in train set never took assesments (only 25% did it) and  only around 21% of ids finished assessments (correct or not). We can train model on these 21% ids.